In [1]:
import time
import datetime
from mkndaq.inst.neph import NEPH

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.0.50',
                    'port': 32783,
                    'timeout': 5,
                },
                # 'set_datetime': '',
                # 'sampling_interval': '1',
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
            },
                'reporting_interval': '10',
                'logs': 'mkndaq/logs',
                'data': 'mkndaq/data',
                'staging': {
                    'path': 'mkndaq/staging',
                },
}
ne300 = NEPH('ne300', cfg)
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

# Initialize NEPH (name: ne300  S/N: 23-0690)


  Instrument identified itself as '{'id': 'ACOEM Aurora NE-300 Sub-Type: 0 Range: 0 Build: 158 Branch: 300'}'.


In [2]:
res = ne300.get_id()
print(res)

{'id': 'ACOEM Aurora NE-300 Sub-Type: 0 Range: 0 Build: 158 Branch: 300'}


In [3]:
# mimick VI099 response
ne300.get_current_data(strict=True, verbosity=0)

{99: '16/05/2024 12:55:16 0.2798123359680176 -0.05397605895996094 0.41529273986816406 -0.3749265670776367 0.7562541961669922 -0.03302192687988281 25.43999671936035 27.719995498657227 7.37000036239624 956.7999877929688 4.203895392974451e-45 0.0'}

In [4]:
ne300.get_current_data(verbosity=0)

{1: datetime.datetime(2024, 5, 16, 12, 55, 20),
 1635000: 2.0939722061157227,
 1525000: -0.04936790466308594,
 1450000: 0.49793052673339844,
 1635090: -0.6608588695526123,
 1525090: 0.3310256004333496,
 1450090: 0.8302202224731445,
 5001: 25.45000648498535,
 5004: 27.7600040435791,
 5003: 7.37000036239624,
 5002: 956.77001953125,
 4036: 4.203895392974451e-45,
 4035: 0.0}

In [5]:
ne300.get_values([1635000, 1525000], verbosity=2)

message sent: b'\x02\x00\x04\x03\x00\x08\x00\x18\xf2\xb8\x00\x17E\x08\x05\x04'
response (bytes): b'\x02\x00\x04\x03\x00\x08\xbf\x96\x038\xbe\xf3\xb4 \xc6\x04'
response length : 2
items: 2
items (bytes): [b'\xbf\x96\x038', b'\xbe\xf3\xb4 ']
response items (bytes):
{1635000: b'\xbf\x96\x038', 1525000: b'\xbe\xf3\xb4 '}
response items:
{1635000: -1.1719732284545898, 1525000: -0.4759836196899414}


{1635000: -1.1719732284545898, 1525000: -0.4759836196899414}

In [6]:
# retrieve VI099 command parameters
ne300.get_values(parameters=[1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035])

{1: datetime.datetime(2024, 5, 16, 12, 55, 43),
 1635000: -1.135981559753418,
 1525000: -0.043723106384277344,
 1450000: 0.9687709808349609,
 1635090: 0.2254951000213623,
 1525090: -0.2645869255065918,
 1450090: -0.18549251556396484,
 5001: 25.469995498657227,
 5004: 27.920007705688477,
 5003: 7.380000114440918,
 5002: 956.7999877929688,
 4036: 4.203895392974451e-45,
 4035: 0.0}

In [7]:
ne300.get_current_data(add_params=[1001, 1002, 1003, 1004, 1005, 2001, 2002])

{1: datetime.datetime(2024, 5, 16, 12, 55, 50),
 1635000: 0.3980731964111328,
 1525000: 1.2457408905029297,
 1450000: 1.1552314758300781,
 1635090: -1.1231684684753418,
 1525090: 0.2552766799926758,
 1450090: 0.1261587142944336,
 5001: 25.469995498657227,
 5004: 27.959985733032227,
 5003: 7.390000343322754,
 5002: 956.780029296875,
 4036: 4.203895392974451e-45,
 4035: 0.0,
 1001: 0.0,
 1002: 0.0,
 1003: 1.401298464324817e-45,
 1004: 2.802596928649634e-45,
 1005: 0.0,
 2001: 1.8216880036222622e-44,
 2002: 8.407790785948902e-44}

In [9]:
# not implemented
ne300.get_new_data()

Not implemented.


''

In [10]:
# not implemented
ne300.get_all_data()

Not implemented. Use 'get_logged_data' with specified period instead.


''

In [11]:
ne300.get_instr_type()

[158, 300, 0, 0]

In [12]:
ne300.get_datetime()

datetime.datetime(2024, 5, 16, 12, 56, 23)

In [13]:
ne300.get_logging_config()

[41,
 2635000,
 2525000,
 2450000,
 2635090,
 2525090,
 2450090,
 5001,
 5002,
 5003,
 5004,
 5005,
 5006,
 5010,
 26635000,
 26525000,
 26450000,
 13525000,
 15635000,
 15525000,
 15450000,
 11635000,
 11525000,
 11450000,
 11635090,
 11525090,
 11450090,
 6007,
 6008,
 6001,
 6002,
 6003,
 6635000,
 6525000,
 6450000,
 6635090,
 6525090,
 6450090,
 5011,
 5012,
 5014,
 4035]

In [ ]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in [1, 2, 0]:
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)

In [64]:
from datetime import datetime, timezone, timedelta
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=10)
print(f"start: {start}\nend  : {end}")
data = ne300.get_logged_data(start=start, end=end, verbosity=1)
# for i in range(len(data)):
    # print(data[i])

start: 2024-05-16 13:46:32.338087+00:00
end  : 2024-05-16 13:56:32.338087+00:00
message sent: b'\x02\x00\x07\x03\x00\x08a`\xdb\xa0a`\xde \x8b\x04'
{2635000: 1.9094690084457397, 2525000: 2.3691649436950684, 2450000: 2.2511560916900635, 2635090: -0.21088840067386627, 2525090: 0.5288919806480408, 2450090: 0.44407370686531067, 5001: 26.990020751953125, 5002: 959.0999755859375, 5003: 7.659999847412109, 5004: 29.8699893951416, 5005: 954.6900024414062, 5006: 36.439998626708984, 5010: 0.0, 26635000: 0.03050420992076397, 26525000: 0.03103429079055786, 26450000: 0.029848219826817513, 13525000: 256.6712951660156, 15635000: 1598518.0, 15525000: 1972750.0, 15450000: 1456574.0, 11635000: 17127.1796875, 11525000: 20480.990234375, 11450000: 20412.4609375, 11635090: 11260.5400390625, 11525090: 13971.98046875, 11450090: 12918.33984375, 6007: 24.177249908447266, 6008: 1570.0269775390625, 6001: 18.435060501098633, 6002: 4.804687976837158, 6003: 20.951459884643555, 6635000: 0.011181330308318138, 6525000: 0

In [66]:
# repeat packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,1]))
response = ne300.tcpip_comm(message=message)
print(response)
display(ne300._acoem_decode_logged_data(response=response))

b'\x02\x00\x07\x03\x07\xe8\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00*\x00(4\xf8\x00&\x87H\x00%bP\x00(5R\x00&\x87\xa2\x00%b\xaa\x00\x00\x13\x89\x00\x00\x13\x8a\x00\x00\x13\x8b\x00\x00\x13\x8c\x00\x00\x13\x8d\x00\x00\x13\x8e\x00\x00\x13\x92\x01\x96j\xf8\x01\x94\xbdH\x01\x93\x98P\x00\xce`\x08\x00\xee\x928\x00\xec\xe4\x88\x00\xeb\xbf\x90\x00\xb1\x898\x00\xaf\xdb\x88\x00\xae\xb6\x90\x00\xb1\x89\x92\x00\xaf\xdb\xe2\x00\xae\xb6\xea\x00\x00\x17w\x00\x00\x17x\x00\x00\x17q\x00\x00\x17r\x00\x00\x17s\x00e=\xf8\x00c\x90H\x00bkP\x00e>R\x00c\x90\xa2\x00bk\xaa\x00\x00\x13\x93\x00\x00\x13\x94\x00\x00\x13\x96\x00\x00\x0f\xc3\x00\x00\x00\x00\x00\x00\x00\x00a`\xdb\xc0\x00\x00\x00<\x00\x00\x00*?\xf4i{@\x17\xa0f@\x10\x12\xf1\xbeW\xf3!?\x07ew>\xe3]\xa1A\xd7\xeb\x90Do\xc6f@\xf5\x1e\xb8A\xee\xf5\xbdDn\xac)B\x11\xc2\x8f\x00\x00\x00\x00<\xf9\xe3\xf7<\xfe;\xa0<\xf4\x84AC\x80U\xedI\xc3!\xb0I\xf0\xd0pI\xb1\xcd\xf0F\x85\xce\\F\xa0\x01\xfbF\x9fx\xecF/\xf2)FZO\xecFI\xd9\\A\xc1k\x02D\xc4@\xddA\x93{\x0

[{2635000: 1.9094690084457397,
  2525000: 2.3691649436950684,
  2450000: 2.2511560916900635,
  2635090: -0.21088840067386627,
  2525090: 0.5288919806480408,
  2450090: 0.44407370686531067,
  5001: 26.990020751953125,
  5002: 959.0999755859375,
  5003: 7.659999847412109,
  5004: 29.8699893951416,
  5005: 954.6900024414062,
  5006: 36.439998626708984,
  5010: 0.0,
  26635000: 0.03050420992076397,
  26525000: 0.03103429079055786,
  26450000: 0.029848219826817513,
  13525000: 256.6712951660156,
  15635000: 1598518.0,
  15525000: 1972750.0,
  15450000: 1456574.0,
  11635000: 17127.1796875,
  11525000: 20480.990234375,
  11450000: 20412.4609375,
  11635090: 11260.5400390625,
  11525090: 13971.98046875,
  11450090: 12918.33984375,
  6007: 24.177249908447266,
  6008: 1570.0269775390625,
  6001: 18.435060501098633,
  6002: 4.804687976837158,
  6003: 20.951459884643555,
  6635000: 0.011181330308318138,
  6525000: 0.010759729892015457,
  6450000: 0.013976999558508396,
  6635090: 0.007293078117072

In [67]:
# next packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,0]))
response = ne300.tcpip_comm(message=message)
response

b'\x02\x00\x07\x03\x00\x00\x06\x04'